### Setup & Data Extraction

In [0]:
import mlflow
from pyspark.sql.functions import col, when
from pyspark.ml.functions import vector_to_array
from pyspark.ml.feature import VectorAssembler

# 1. Set Governance Context 
spark.sql("USE CATALOG ecommerce_capstone")
spark.sql("USE SCHEMA churn_analysis")

# 2. Load Model with Volume Path 
volume_path = "/Volumes/ecommerce_capstone/churn_analysis/model_storage"
manual_run_id = "9b9d1c8c8838499c85c0010a05a4a661" 
model_uri = f"runs:/{manual_run_id}/churn_model"

loaded_model = mlflow.spark.load_model(model_uri=model_uri, dfs_tmpdir=volume_path)

# 3. Load and Prepare Data (Requirement: Feature Extraction) 
gold_data = spark.read.table("gold_user_features")

# Re-create the 'features' vector that the model expects 
feature_cols = ["Frequency", "Monetary", "Recency"]
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
data_with_features = assembler.transform(gold_data)

# 4. Apply Model 
predictions = loaded_model.transform(data_with_features)

# 5. Innovation: Generate Actionable Recommendations 
final_output = predictions.withColumn("churn_probability", vector_to_array("probability")[1]) \
    .withColumn(
        "Recommended_Action",
        when(col("churn_probability") > 0.8, "High Risk: Send 25% Discount")
        .when(col("churn_probability") > 0.5, "Medium Risk: Send Loyalty Email")
        .otherwise("Low Risk: Standard Marketing")
    )

# 6. Store Predictions Back to Database 
final_output.select("Customer_ID", "Recency", "churn_probability", "Recommended_Action") \
    .write.format("delta").mode("overwrite") \
    .saveAsTable("final_customer_retention_plan")

display(spark.read.table("final_customer_retention_plan").limit(10))

Customer_ID,Recency,churn_probability,Recommended_Action
17838.0,5189,1.0,High Risk: Send 25% Discount
12782.0,5167,1.0,High Risk: Send 25% Discount
12408.0,5195,1.0,High Risk: Send 25% Discount
18071.0,5420,1.0,High Risk: Send 25% Discount
13641.0,5803,1.0,High Risk: Send 25% Discount
14364.0,5271,1.0,High Risk: Send 25% Discount
15512.0,5318,1.0,High Risk: Send 25% Discount
13617.0,5203,1.0,High Risk: Send 25% Discount
17328.0,5836,1.0,High Risk: Send 25% Discount
16547.0,5709,1.0,High Risk: Send 25% Discount


The model transforms raw churn probabilities into Recommended Actions. For example, customers with a probability > 0.8 are flagged for a 25% discount. This makes the AI results actionable for the marketing team.